# <p style="text-align: center;">RNA Knowledge Graph Build Data Preparation</p>
    
***
***

**Authors:** [ECavalleri](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=emanuele.cavalleri@unimi.it), [TJCallahan](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=callahantiff@gmail.com), [MMesiti](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=marco.mesiti@unimi.it)

**GitHub Repositories:** [testRNA-KG](https://github.com/emanuelecavalleri/testRNA-KG), [PheKnowLator](https://github.com/callahantiff/PheKnowLator/)  
<!--- **Release:** **[v2.0.0](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)** --->
  
<br>  
  
**Purpose:** This notebook serves as a script to download, process, map, and clean data in order to build edges for the simplified RNA-centered Knowledge Graph.

<br>

**Assumptions:**   
- Edge data downloads ➞ `./resources/edge_data`  
- Ontologies ➞ `./resources/ontologies`    
- Processed data write location ➞ `./resources/processed_data`  

<br>

**Dependencies:**   
- **Scripts**: This notebook utilizes several helper functions, which are stored in the [`data_utils.py`](https://github.com/callahantiff/PheKnowLator/blob/master/pkt_kg/utils/data_utils.py) and [`kg_utils.py`](https://github.com/callahantiff/PheKnowLator/blob/master/pkt_kg/utils/kg_utils.py) scripts.  
- **Data**: All downloaded and generated data sources are provided through [this](https://drive.google.com/drive/folders/1sev5zczMviX7UVqMhTpkFXG43K3nQa9f) dedicated Google Drive repository. <u>This notebook will download everything that is needed for you</u>.  
_____
***

## Table of Contents
***

### [Download Ontologies](#create-ontologies)


### [Create Identifier Maps ](#create-identifier-maps)   


### [Download and process Edge Datasets](#create-edges)  

____
***

## Set-Up Environment
***

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt
sys.path.append('../')

In [ ]:
# import needed libraries
import datetime
import glob
import itertools
import networkx
import numpy
import os
import openpyxl
import pandas
import pickle
import re
import requests
import sys

from collections import Counter
from functools import reduce
from rdflib import Graph, Namespace, URIRef, BNode, Literal
from rdflib.namespace import OWL, RDF, RDFS
from reactome2py import content
from tqdm import tqdm
from typing import Dict
from typing import Tuple

from pkt_kg.utils import *

import pandas as pd
import gffpandas.gffpandas as gffpd
import tarfile

#### Define Global Variables

In [ ]:
# directory to store resources
resource_data_location = '../resources/'

# directory to use for processing data
processed_data_location = '../resources/processed_data/'

# directory to write ontology data to
ontology_data_location = '../resources/ontologies/'

# directory to write edges data to
edge_data_location = '../resources/edge_data/'

# owltools location
owltools_location = '../pkt_kg/libs/owltools'

In [ ]:
# Download data function for already processed data
def download(name, path):
    url = 'https://storage.googleapis.com/pheknowlator/current_build/data/processed_data/'+name
    #if not os.path.exists(path + name):
    data_downloader(url, path)

***
***
### DOWNLOAD ONTOLOGIES  <a class="anchor" id="create-ontologies"></a>
We must establish a unified standard for identifying entities within our simplified RNA-centered KG. Entities are relations, diseases, miRNAs, and genes. While well-reputed bio-ontologies provide terms for relations and diseases, miRNAs and genes lack direct correspondences.
***
***

### Relation Ontology ([RO](https://www.ebi.ac.uk/ols/ontologies/ro))
The OBO Relations Ontology (RO) is a collection of OWL relations (ObjectProperties) intended for use across a wide variety of biological ontologies.

In [ ]:
if not os.path.exists(ontology_data_location + 'ro_with_imports.owl'):
    command = '{} {} --merge-import-closure -o {}'
    os.system(command.format(owltools_location, 'http://purl.obolibrary.org/obo/ro.owl',
                             ontology_data_location + 'ro_with_imports.owl'))

In [ ]:
# Relations labels are already provided by PKL ecosystem
download('RELATIONS_LABELS.txt', '../resources/relations_data/')
download('INVERSE_RELATIONS.txt', '../resources/relations_data/')

# Load data, print row count, and preview it
ro_data_label = pandas.read_csv('../resources/relations_data/'+'RELATIONS_LABELS.txt', header=0, delimiter='\t')

print('There are {edge_count} RO Relations and Labels'.format(edge_count=len(ro_data_label)))
ro_data_label.head(n=5)

***
### Mondo Disease Ontology ([Mondo](https://www.ebi.ac.uk/ols/ontologies/ro))
A semi-automatically constructed ontology that merges in multiple disease resources to yield a coherent merged ontology.

In [ ]:
if not os.path.exists(ontology_data_location + 'mondo_with_imports.owl'):
    command = '{} {} --merge-import-closure -o {}'
    os.system(command.format(owltools_location, 'http://purl.obolibrary.org/obo/mondo.owl',
                             ontology_data_location + 'mondo_with_imports.owl'))

At this point, please run the [<tt>Ontology_Cleaning.ipynb</tt>](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Ontology_Cleaning.ipynb) notebook provided by PKT.

***
***
### DOWNLOAD EDGES  <a class="anchor" id="create-edges"></a>
***
***

### gene-disease from [Human Disease Molecular Mechanisms](https://github.com/callahantiff/PheKnowLator/wiki/Building-a-KG-of-Human-Disease-Molecular-Mechanisms) (PKT-built)

In [ ]:
data_downloader('https://storage.googleapis.com/pheknowlator/current_build/data/original_data/curated_gene_disease_associations.tsv',
                edge_data_location)

# Rename file adding relationship's identifier
os.rename(edge_data_location+'curated_gene_disease_associations.tsv',
          edge_data_location+'gene-disease_curated_gene_disease_associations.tsv')

with open(edge_data_location + 'gene-disease_curated_gene_disease_associations.tsv') as f:
    data = f.read()

data = pd.read_csv(edge_data_location + 'gene-disease_curated_gene_disease_associations.tsv', sep="\t")  
data

In [ ]:
data = data.sample(n=50000)
data.to_csv(edge_data_location + 'gene-disease_curated_gene_disease_associations.tsv', header=None, sep='\t', index=None)

For representing genes, we can use NCBI Entrez Gene identifiers (<tt>geneID</tt> column), , and it's worth noting that symbols could have been a viable choice as well. For denoting diseases (<tt>diseaseID</tt> column), we can notice the original tsv adopts DisGeNET identifiers. We'll need to establish a mapping that links these identifiers to the Mondo ontology.

***
### gene-miRNA from [TarBase](https://dianalab.e-ce.uth.gr/html/diana/web/index.php?r=tarbasev8/index)
DIANA-TarBase v8 is a reference database devoted to the indexing of experimentally supported microRNA (miRNA) targets.

In [ ]:
data_downloader('https://dianalab.e-ce.uth.gr/downloads/tarbase_v8_data.tar.gz', edge_data_location)

with tarfile.TarFile(edge_data_location+'tarbase_v8_data.tar', 'r') as tar_ref:
    tar_ref.extractall(edge_data_location)
    
# Remove tar file
os.remove(edge_data_location+'tarbase_v8_data.tar')
    
# Rename file adding relationship's identifier
os.rename(edge_data_location+'TarBase_v8_download.txt',
          edge_data_location+'gene-miRNA_TarBase_v8_download.txt')   

with open(edge_data_location + 'gene-miRNA_TarBase_v8_download.txt') as f:
    data = f.read()

data = pd.read_csv(edge_data_location + 'gene-miRNA_TarBase_v8_download.txt', sep="\t", dtype={"cell_line": "string"})  
data

In [ ]:
# For the time being, we keep only Homo sapiens rows
data = data[data['species'].str.contains("Homo sapiens")]

# Moreover, we keep only 50k (random) rows to reduce input size
data = data.sample(n=50000)

# This simplified KG ignores if a transcript is "3p" or "5p", so we store this information as additional column
data['p'] = data[data['mirna'].str.contains("p")]['mirna']
data["p"] = data["p"].str[-2:]
data['mirna'] = data['mirna'].str.replace(r'-[35]p$', '', regex=True)
data['mirna'] = data['mirna'].str.lower()

# If you're interested in understanding why Homo sapiens has -3p and -5p miRNAs:
# https://pubmed.ncbi.nlm.nih.gov/12592000/
# Putting aside the -(3/5)p information, we are essentially dealing with non-mature (aka hairpin) miRNA:
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3616697/

data

In this case, we have only symbols and ENSG identifiers (<tt>geneId</tt> and <tt>geneName</tt> columns) for identifying genes, we'll require an additional mapping to link these symbols or ENSG to NCBI Entrez Gene identifiers.

In [ ]:
data.to_csv(edge_data_location + 'gene-miRNA_TarBase_v8_download.txt', header=None, sep='\t', index=None)

***
### miRNA-disease from [miR2Disease](http://watson.compbio.iupui.edu:8080/miR2Disease/)
miR2Disease, a manually curated database, aims at providing a comprehensive resource of miRNA deregulation in various human diseases.

In [ ]:
data_downloader('http://watson.compbio.iupui.edu:8080/miR2Disease/download/AllEntries.txt', edge_data_location)

data = pd.read_csv(edge_data_location + 'AllEntries.txt', sep="\t", header=None)  
os.remove(edge_data_location + 'AllEntries.txt')
data

In [ ]:
# miR2Disease provides a look-up table for mapping disease names to DO terms
data_downloader('http://watson.compbio.iupui.edu:8080/miR2Disease/download/diseaseList.txt', processed_data_location)

descDOmap = pd.read_csv(processed_data_location + 'diseaseList.txt', sep="\t")  
os.remove(processed_data_location + 'diseaseList.txt')
descDOmap

In [ ]:
# Ontologies are represented in OWL files that make use of _ for URIs
descDOmap['disease ontology ID'] = descDOmap['disease ontology ID'].astype(str).str.replace(':', '_')

disease2mirna = pd.merge(descDOmap, data, left_on=['disease name in original paper'], right_on=[1]).drop(
    columns=['disease name in original paper'])
disease2mirna[0] = disease2mirna[0].str.lower()
disease2mirna

We lost 2,902-2,624=278 rows during mapping (278/2,902<10%).

In [ ]:
disease2mirna.to_csv(edge_data_location + 'miRNA-disease_miR2Disease.txt', header=None, sep='\t', index=None)

***
***
### CREATE MAPPING DATASETS  <a class="anchor" id="create-identifier-maps"></a>
***
***

### Ensembl Gene-Entrez Gene <a class="anchor" id="ensemblgene-entrezgene"></a>


**Purpose:** To map Ensembl gene identifiers to Entrez gene identifiers

**Output:** `ENSEMBL_GENE_ENTREZ_GENE_MAP.txt`

Already provided by PKL ecosystem.

In [ ]:
download('ENSEMBL_GENE_ENTREZ_GENE_MAP.txt', processed_data_location)

ensEntrez = pd.read_csv(processed_data_location + 'ENSEMBL_GENE_ENTREZ_GENE_MAP.txt', sep="\t", header=None)
ensEntrez

***
### DisGeNET-Mondo <a class="anchor" id="DisGeNET-Mondo"></a>


**Purpose:** To map DisGeNET identifiers to Mondo identifiers

**Output:** `DISEASE_MONDO_MAP.txt`

Already provided by PKL ecosystem.

In [ ]:
download('DISEASE_MONDO_MAP.txt', processed_data_location)

disMondo = pd.read_csv(processed_data_location + 'DISEASE_MONDO_MAP.txt', sep="\t", header=None)
disMondo

***
### Disease Ontology (DO) - Mondo mapping <a class="anchor" id="ensemblgene-entrezgene"></a>


**Purpose:** To map DO identifiers to Mondo identifiers

**Output:** `DISEASE_DOID_MONDO_Map.txt`

In [ ]:
mondo_graph = Graph().parse(ontology_data_location + 'mondo_with_imports.owl')

dbxref_res = gets_ontology_class_dbxrefs(mondo_graph)[0]

# Fix DOIDs (substitute : with _) and upper case them
mondo_dict = {str(k).replace(':','_').upper(): {str(i).split('/')[-1].replace(':','_') for i in v}
              for k, v in dbxref_res.items() if 'doid' in str(k)}

with open(processed_data_location + 'DISEASE_DOID_MONDO_Map.txt', 'w') as outfile:
    for k, v in mondo_dict.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')
 
doidMondo = pd.read_csv(processed_data_location + 'DISEASE_DOID_MONDO_Map.txt', sep="\t", header=None)
doidMondo[1] = doidMondo[1].str.split(',')
doidMondo = doidMondo.explode(1)
doidMondo.to_csv(processed_data_location + 'DISEASE_DOID_MONDO_Map.txt', header=None, sep='\t', index=None)

In [ ]:
doidMondo = pd.read_csv(processed_data_location + 'DISEASE_DOID_MONDO_Map.txt', sep="\t", header=None)
doidMondo

***
### miRBase ID - miRBase accession


**Purpose:** To map miRNA identifiers from miRBase to miRBase accession (it guarantees a standard identification of miRNA molecules via URIs)

**Output:** `MIRBASE_ID_ACCESSION_MAP.txt`

In [ ]:
data_downloader('https://www.mirbase.org/download/hsa.gff3', processed_data_location)

miRBaseMap = gffpd.read_gff3(processed_data_location + 'hsa.gff3')  
os.remove(processed_data_location + 'hsa.gff3')
print(miRBaseMap.header)
print(miRBaseMap.df)

In [ ]:
miRBaseMap = miRBaseMap.attributes_to_columns()
miRBaseMap = miRBaseMap[['attributes']]
miRBaseMap

In [ ]:
miRBaseMap = miRBaseMap.attributes.str.split(';',expand=True)
# Keep only "ID" and "Name" columns
miRBaseMap = miRBaseMap[[2,0]]
# Remove substring "ID="
miRBaseMap[0] = miRBaseMap[0].str[3:]
# Remove substring "Name="
miRBaseMap[2] = miRBaseMap[2].str[5:]
# Keep only hairpin/stem-loop miRNAs
# (those starting with MI and not MIMAT, last one is reserved for mature sequences)
miRBaseMap = miRBaseMap[~miRBaseMap[0].str.startswith('MIMAT')]
miRBaseMap

In [ ]:
miRBaseMap.to_csv(processed_data_location+'MIRBASE_ID_ACCESSION_MAP.txt', header=None,sep='\t', index=None)

***
To represent genes, PKT designates them as subclasses of relevant Sequence Ontology ([SO](https://www.ebi.ac.uk/ols4/ontologies/so)) terms. We add miRNAs as subclasses of [SO_0000647](http://purl.obolibrary.org/obo/SO_0000647) (*miRNA_primary_transcript*).

In [ ]:
# KG construction approach dictionary (for non ontological data), provided by PKL ecosystem
download('subclass_construction_map.pkl', '../resources/construction_approach/')

# Load data, print row count, and preview it
nonO_data = pd.read_pickle(r'../resources/construction_approach/subclass_construction_map.pkl')

# For instance, ncbi IDs are mapped to appropriate SO Ontology entries
list(nonO_data.items())[:5]

In [ ]:
miRBaseMap['SO'] = [['SO_0000647']] * len(miRBaseMap)

mirna_nonO = miRBaseMap.drop(2, axis=1).set_index(0).to_dict()
nonO_data = {**nonO_data, **mirna_nonO['SO']}

list(nonO_data.items())[len(list(nonO_data.items()))-5:len(list(nonO_data.items()))]

In [ ]:
with open('../resources/construction_approach/subclass_construction_map.pkl', 'wb') as handle:
    pickle.dump(nonO_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

***
PKT also provides node and relation metadata (a.k.a. node properties and relation attributes).

In [ ]:
# KG metadata, provided by PKL ecosystem
download('node_metadata_dict.pkl', '../resources/node_data/')

# Load data, print row count, and preview it
metadata = pd.read_pickle(r'../resources/node_data/node_metadata_dict.pkl')

metadata.keys()

In [ ]:
{k: metadata['nodes'][k] for k in list(metadata['nodes'])[:5]}

In [ ]:
{k: metadata['relations'][k] for k in list(metadata['relations'])[:5]}

We can add miRNA properties (*Label*, *Description*, and *Synonym*) from miRBase.

In [ ]:
from Bio import SeqIO

data_downloader('https://www.mirbase.org/download/miRNA.dat', processed_data_location)

# Open the EMBL file
embl_file = processed_data_location + 'miRNA.dat'

# Create empty lists to store the data
data = {
    "ID": [],
    "Description": [],
    "Sequence": [],
    "Comments": [],
    "References": []
}

# Iterate through the records in the EMBL file
for record in SeqIO.parse(embl_file, "embl"):
    data["ID"].append(record.id)
    data["Description"].append(record.description)
    data["Sequence"].append(str(record.seq))
    data["Comments"].append(str(record.annotations.get('comment', '')))
    references = []
    i = 0
    for ref in record.annotations.get('references', []):
        i = i + 1
        references.append(f"{[i], ref.pubmed_id}")
    data["References"].append(", ".join(references))

df = pd.DataFrame(data)
df

In [ ]:
# Reduce to "ID", "Description", and "Synonym" columns
df = df[df['Description'].astype(str).str.contains('Homo sapiens')]
df['Synonym'] = df['Description']
df['Description'] = df['Comments'].astype(str) + df['References'].astype(str) + '. Sequence: ' + df['Sequence'].astype(str)
df = df[['ID', 'Description', 'Synonym']]
df

Finally, we add *Labels* through the map we have previously developed.

In [ ]:
df = pd.merge(df, miRBaseMap, left_on=['ID'], right_on=[0])
df['Label'] = df[2]
df = df[['ID', 'Label', 'Description', 'Synonym']]
df

In [ ]:
# Convert the DataFrame to dictionary format
miRNA_dict = {}
for index, row in df.iterrows():
    # Transform the ID to correspondent URI (KG's ID)
    gene_id = f'https://www.mirbase.org/hairpin/{row["ID"]}'
    miRNA_dict[gene_id] = {
        'Label': row['Label'],
        'Description': row['Description'],
        'Synonym': row['Synonym']
    }
    
{k: miRNA_dict[k] for k in list(miRNA_dict)[:5]}

In [ ]:
nodes_dict = {**metadata.get('nodes'), **miRNA_dict}
nodes_final_dict = {'nodes': nodes_dict}

rel_dict = {'relations': {**metadata.get('relations')}}
metadata = {**nodes_final_dict, **rel_dict}

with open('../resources/node_data/node_metadata_dict.pkl', 'wb') as handle:
    pickle.dump(metadata, handle, protocol=pickle.HIGHEST_PROTOCOL)

***
If you wish to include additional bio-entities to enhance this KG, you will need to expand this dictionary. If you are interested, you can explore the entire [RNA-KG](https://github.com/AnacletoLAB/RNA-KG/). Feel free to contact me at [emanuele dot cavalleri at unimi dot it](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=emanuele.cavalleri@unimi.it).